In [1]:
import cudf, cuml, cupy, dask_cudf    
from cuml.metrics import accuracy_score as cuml_accuracy_score
from cuml.dask.common.utils import persist_across_workers
import xgboost
import dask
import time, sys, os, argparse
import glob, json, pprint, joblib
from dask.distributed import wait, Client
import warnings; warnings.filterwarnings("ignore")

from dask_ml.model_selection import train_test_split
from cuml.dask.common.utils import persist_across_workers

In [3]:
from dask.distributed import Client
client = Client('a3d51b0a71fb14120825543f299d328e-83594755.us-west-2.elb.amazonaws.com:8786')
client.restart()

Client Scheduler: tcp://a3d51b0a71fb14120825543f299d328e-83594755.us-west-2.elb.amazonaws.com:8786 Dashboard: /proxy/rapidsai-scheduler:8787/status,Cluster Workers: 7 Cores: 42 Memory: 350.00 GB


In [4]:
# Query the client for all connected workers
workers = client.has_what().keys()
n_workers = len(workers)
n_streams = 8 # Performance optimization
worker_list = list(workers)
display(worker_list)

['tcp://192.168.17.137:35391',
 'tcp://192.168.19.182:39897',
 'tcp://192.168.28.129:33319',
 'tcp://192.168.4.228:33047',
 'tcp://192.168.70.245:35999',
 'tcp://192.168.72.160:35135',
 'tcp://192.168.93.101:45601']

In [5]:
from s3fs import S3FileSystem
s3 = S3FileSystem(anon=True)
s3.ls('s3://sagemaker-rapids-hpo-us-west-2/10_year/')

['sagemaker-rapids-hpo-us-west-2/10_year/_common_metadata',
 'sagemaker-rapids-hpo-us-west-2/10_year/_metadata',
 'sagemaker-rapids-hpo-us-west-2/10_year/part.0.parquet',
 'sagemaker-rapids-hpo-us-west-2/10_year/part.1.parquet',
 'sagemaker-rapids-hpo-us-west-2/10_year/part.10.parquet',
 'sagemaker-rapids-hpo-us-west-2/10_year/part.100.parquet',
 'sagemaker-rapids-hpo-us-west-2/10_year/part.101.parquet',
 'sagemaker-rapids-hpo-us-west-2/10_year/part.102.parquet',
 'sagemaker-rapids-hpo-us-west-2/10_year/part.103.parquet',
 'sagemaker-rapids-hpo-us-west-2/10_year/part.104.parquet',
 'sagemaker-rapids-hpo-us-west-2/10_year/part.105.parquet',
 'sagemaker-rapids-hpo-us-west-2/10_year/part.106.parquet',
 'sagemaker-rapids-hpo-us-west-2/10_year/part.107.parquet',
 'sagemaker-rapids-hpo-us-west-2/10_year/part.108.parquet',
 'sagemaker-rapids-hpo-us-west-2/10_year/part.109.parquet',
 'sagemaker-rapids-hpo-us-west-2/10_year/part.11.parquet',
 'sagemaker-rapids-hpo-us-west-2/10_year/part.110.par

In [6]:
dataset_feature_columns = [ 'Year', 'Quarter', 'Month', 'DayOfWeek', 
                            'Flight_Number_Reporting_Airline', 'DOT_ID_Reporting_Airline',
                            'OriginCityMarketID', 'DestCityMarketID',
                            'DepTime', 'DepDelay', 'DepDel15', 'ArrDel15',
                            'AirTime', 'Distance' ]

# target_files = glob.glob('datasets/1_year/*.parquet')
# n_datafiles = len(target_files)
# dataset = dask_cudf.read_parquet(target_files, dataset_feature_columns)

dataset = dask_cudf.read_parquet('s3://sagemaker-rapids-hpo-us-west-2/10_year/*.parquet',storage_options={'anon':True})
dataset = dataset.dropna()

In [7]:
display(dataset)
print(len(dataset))

,Year,Quarter,Month,DayOfWeek,DOT_ID_Reporting_Airline,Flight_Number_Reporting_Airline,OriginCityMarketID,DestCityMarketID,DepTime,DepDelay,DepDel15,ArrTime,ArrDelay,ArrDel15,AirTime,Distance
npartitions=121,,,,,,,,,,,,,,,,
,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float32,float32,int32,float32,float32
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


62992062


In [8]:
# split [ always runs, regardless of whether dataset is cached ]
train, test = train_test_split( dataset, random_state = 42 ) 

In [9]:
# build X [ features ], y [ labels ] for the train and test subsets
label_column = 'ArrDel15'
y_train = train[label_column]; 
X_train = train.drop(label_column, axis = 1)
y_test = test[label_column]
X_test = test.drop(label_column, axis = 1)

In [10]:
%%time
X_train, y_train, X_test, y_test = persist_across_workers(client, [X_train, y_train, X_test, y_test], 
                                                          workers = worker_list)
wait([X_train, y_train, X_test, y_test ])

CPU times: user 493 ms, sys: 33.3 ms, total: 527 ms
Wall time: 2.83 s


DoneAndNotDoneFutures(done={<Future: finished, type: cudf.DataFrame, key: ('drop_by_shallow_copy-2e45b2ae6b31921eb9fc2fb2e4ff1f8f', 6)>, <Future: finished, type: cudf.DataFrame, key: ('drop_by_shallow_copy-e52226113ed9ecf81da80a6e5fe69283', 17)>, <Future: finished, type: cudf.DataFrame, key: ('drop_by_shallow_copy-e52226113ed9ecf81da80a6e5fe69283', 80)>, <Future: finished, type: cudf.DataFrame, key: ('drop_by_shallow_copy-2e45b2ae6b31921eb9fc2fb2e4ff1f8f', 69)>, <Future: finished, type: cudf.DataFrame, key: ('drop_by_shallow_copy-e52226113ed9ecf81da80a6e5fe69283', 95)>, <Future: finished, type: cudf.Series, key: ('getitem-6d1cb3c598cd566331dd682f473cc534', 101)>, <Future: finished, type: cudf.Series, key: ('getitem-6d1cb3c598cd566331dd682f473cc534', 82)>, <Future: finished, type: cudf.Series, key: ('getitem-d60bdf70c9f37c4da4c76120df3f21c2', 31)>, <Future: finished, type: cudf.DataFrame, key: ('drop_by_shallow_copy-2e45b2ae6b31921eb9fc2fb2e4ff1f8f', 57)>, <Future: finished, type: cudf.

In [11]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('int32')
y_test = y_test.astype('int32')  

In [12]:
%%time
from cuml.dask.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators = 1000,
                                   max_depth = 15,
                                   max_features = 0.9,
                                   n_bins = 32)

trained_model = rf_model.fit( X_train, y_train )

y_test = y_test.compute()
predictions = trained_model.predict( X_test ).compute()
test_accuracy = cuml_accuracy_score ( y_test, predictions )

CPU times: user 1.72 s, sys: 2.42 s, total: 4.15 s
Wall time: 38.4 s


In [13]:
test_accuracy

0.9911624193191528